In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
Qinfo = pd.read_csv('data/QInfo.csv')
Qinfo.head()

In [ ]:
Qbody = pd.read_csv('data/Qbody.csv')
Qbody

In [ ]:
Qbody.info()

In [ ]:
sample_html = Qbody['Body'][442]
sample_html

In [ ]:
soup = BeautifulSoup(sample_html, 'html.parser')
print(soup.text)

In [ ]:
def parse_question(question_html):
    soup = BeautifulSoup(question_html, 'html.parser')
    txt = soup.text
    match = re.search(r'(.+?\nExample)', txt, re.DOTALL)
    txt = match.group(1)
    txt = txt.replace('-', ' ')
    txt = txt.replace('\n', ' ')
    return re.sub(r'[^\w\s]', '', txt).rstrip().lower()
print(parse_question(Qbody['Body'][0]))

In [ ]:
Qbody['text'] = Qbody['Body'].apply(parse_question)

In [ ]:
vocab = set()
def update_set(txt):
    vocab.update(txt.split())
Qbody['text'].apply(update_set)
len(vocab)

In [ ]:
list(vocab)[:10]

In [ ]:
df = pd.DataFrame(
    {
        'word' : list(vocab)
    }
)
df.head()

In [ ]:
def term_freq_fnc(doc):
    def term_freq(word):
        doc_words = doc.split()
        return (doc_words.count(word)/len(doc_words))
    return term_freq

In [ ]:
term_freq = term_freq_fnc(Qbody['text'][0])
tmp_df = df['word'].apply(term_freq)

In [ ]:
tmp_df[tmp_df>0][:10]

In [ ]:
for i, doc in enumerate(Qbody['text']):
    term_freq = term_freq_fnc(doc)
    df[f'TF{i+1}'] = df['word'].apply(term_freq)

In [ ]:
no_of_docs = len(Qbody['text'])
no_of_docs

In [ ]:
tf_cols = [f'TF{i+1}' for i in range(no_of_docs)]
doc_freq = np.sum( (df[tf_cols] > 0).to_numpy(), axis=1, keepdims=True)
doc_freq

In [ ]:
idf = np.log(no_of_docs / doc_freq)
idf

In [ ]:
tf = df[tf_cols].to_numpy()
tf

In [ ]:
tf_idf = tf * idf

In [ ]:
tf_idf

In [ ]:
tmp = tf_idf.sum(axis=1)
np.where(tmp <= 0)

In [ ]:
df['word'][438]

In [ ]:
df

In [ ]:
tfidf_DF = pd.DataFrame(
    tf_idf,
    columns=[f'TF_IDF{i+1}' for i in range(no_of_docs)]
)
tfidf_DF['word'] = df['word']
tfidf_DF

In [ ]:
tfidf_DF.iloc[438]

In [ ]:
tfidf_DF.drop(438, inplace=True)    # example shows a bug

In [ ]:
tfidf_DF

In [ ]:
tfidf_DF.to_csv('tf_idf.csv', index=False)